In [1]:
import logging
from telegram.ext import CommandHandler
from telegram.ext import Updater
from telegram.ext import MessageHandler, Filters
from telegram import *
from telegram.ext import CallbackQueryHandler
import pickle
import random

In [2]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [3]:
updater = Updater(token=pickle.load(open("token", "rb")))

2018-03-01 14:55:23,626 - telegram.ext.dispatcher - DEBUG - Setting singleton dispatcher as <telegram.ext.dispatcher.Dispatcher object at 0x7fef76b9e390>


In [4]:
is_previous_message_question = 0

In [5]:
my_chat_id = pickle.load(open("my_chat_id", "rb"))

In [6]:
tasks = []

In [7]:
add_task = 0

In [8]:
def start(bot, update):
    text = random.choice(["Ага, ну здарова.", "Ку!", "Приветище!", "Ну чего тебе надо?"])
    bot.send_message(chat_id=update.message.chat_id, text=text)
    bot.send_message(chat_id=update.message.chat_id, text="Ну и что ты сделала за сегодня?")
    is_previous_message_question = 1
        
def echo(bot, update):
#     if (is_previous_message_question):
#     if (update.message.text == "1. Пожарить картошку"):
#         key_button1 = KeyboardButton(text="Точно?")
#         keyboard1 = ReplyKeyboardMarkup([[key_button1]])
#         bot.send_message(chat_id=update.message.chat_id, text="Уверена?", reply_markup=keyboard1)
    if add_task == 1:
        add_task = 0
        tasks.append(update.message.text)
        bot.send_message(chat_id=update.message.chat_id, text="Добавил \"" + update.message.text + "\"")
    else:
        if ("ничего" or "нифига" or "нихуя" in update.message.text.lower()):
            text = random.choice(["Ну молодец, молодец! Тебе же нечего делать, да?", "Капец, ты начнешь развиваться, инфузория?", "Мда, капец...", "И чо, тебе норм?..", "Ну да .-.", "Жесть .-."])
            bot.send_message(chat_id=update.message.chat_id, text=text)
        elif len(update.message.text.split()) < 10:
            text = random.choice(["То есть, ничего?..", "Чот как-то не оч .-.", "Мдааа, такое..."])
            bot.send_message(chat_id=update.message.chat_id, text=text)
        elif len(update.message.text.split()) >= 10:
            text = random.choice(["Лол, кек, чебурек)) Надо еще кой-чо сделать.", "Лан, сегодня норм. Но вообще-то у тебя еще есть задачи.", "Ну, неплохо, да, ты вумница :3 Только сделай тут еще один пунктик, плз!"])
            bot.send_message(chat_id=update.message.chat_id, text=text)
        is_previous_message_question = 0
#     else:
#         bot.send_message(chat_id=update.message.chat_id, text="Ничонипонял")
        
def show_keyboard(bot, update):
    if len(tasks) == 0:
        bot.send_message(chat_id=update.message.chat_id, text="Упс, ты молодец, у тебя нет задач :)")
#         bot.send_message(chat_id=update.message.chat_id, text="Лови список задач!", reply_markup=keyboard)
        key_buttons = []
        for i in range(len(tasks)):
            key_buttons.append(InlineKeyboardButton(text=str(i+1) + ". " + tasks[i], callback_data=str(i)))
        key_buttons.append(InlineKeyboardButton(text="Добавить", callback_data=str(-1)))
        keyboard = InlineKeyboardMarkup([key_buttons])
        bot.send_message(chat_id=update.message.chat_id, text="key", reply_markup=keyboard)
    
def callback_query(bot, update):
#     bot.send_message(chat_id=my_chat_id, text=update.callback_query.message.message_id)
    update.callback_query.message.editMessageText(text="jkjkjkj")
    if (int(update.callback_query.data) >= 0):
        key_buttons = []
        key_buttons.append(InlineKeyboardButton(text="Случайно тыкнула!", callback_data=str(-2)))
        key_buttons.append(InlineKeyboardButton(text="Сделала :)", callback_data=str(-3)))
        key_buttons.append(InlineKeyboardButton(text="Просто удали", callback_data=str(-4)))
        keyboard = InlineKeyboardMarkup([key_buttons])
        bot.send_message(chat_id=update.message.chat_id, text="ere", reply_markup=keyboard)        
    elif int(update.callback_query.data) == -1:
        add_task = 1
    bot.send_message(chat_id=my_chat_id, text=update.callback_query.data)

In [9]:
start_handler = CommandHandler('start', start)
show_keyboard_handler = CommandHandler('list', show_keyboard)

In [10]:
callback_query_handler = CallbackQueryHandler(callback_query)

In [11]:
dispatcher = updater.dispatcher
dispatcher.add_handler(start_handler)
dispatcher.add_handler(show_keyboard_handler)
dispatcher.add_handler(callback_query_handler)

In [12]:
echo_handler = MessageHandler(Filters.text, echo)
dispatcher.add_handler(echo_handler)

In [13]:
updater.start_polling()

2018-03-01 14:55:26,001 - JobQueue - DEBUG - JobQueue thread started
2018-03-01 14:55:26,005 - telegram.ext.updater - DEBUG - dispatcher - started
2018-03-01 14:55:26,006 - telegram.ext.updater - DEBUG - updater - started
2018-03-01 14:55:26,012 - telegram.ext.dispatcher - DEBUG - Dispatcher started
2018-03-01 14:55:26,012 - telegram.ext.updater - DEBUG - Updater thread started


2018-03-01 14:55:26,026 - telegram.bot - DEBUG - Entering: set_webhook
2018-03-01 14:55:26,029 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): api.telegram.org
2018-03-01 14:55:26,817 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - https://api.telegram.org:443 "POST /bot484949622:AAGVa89zUGszMMp9D4Y48k-eFcw3xyQPukQ/setWebhook HTTP/1.1" 200 68
2018-03-01 14:55:26,822 - telegram.bot - DEBUG - True
2018-03-01 14:55:26,826 - telegram.bot - DEBUG - Exiting: set_webhook
2018-03-01 14:55:26,831 - telegram.bot - DEBUG - Entering: get_updates
2018-03-01 14:55:30,273 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - DEBUG - https://api.telegram.org:443 "POST /bot484949622:AAGVa89zUGszMMp9D4Y48k-eFcw3xyQPukQ/getUpdates HTTP/1.1" 200 656
2018-03-01 14:55:30,279 - telegram.bot - DEBUG - Getting updates: [976005332]
2018-03-01 14:55:30,284 - telegram.bot - DEBUG - [<telegram.update.Update object at 0x7fef76bc1630>]
2018-03-01 14:5

In [14]:
updater.stop()

2018-03-01 14:48:55,878 - JobQueue - DEBUG - JobQueue thread stopped
2018-03-01 14:48:55,882 - telegram.ext.updater - DEBUG - Stopping Updater and Dispatcher...
2018-03-01 14:48:55,884 - telegram.ext.updater - DEBUG - Requesting Dispatcher to stop...
2018-03-01 14:48:56,040 - telegram.ext.dispatcher - DEBUG - orderly stopping
2018-03-01 14:48:56,043 - telegram.ext.dispatcher - DEBUG - Dispatcher thread stopped
2018-03-01 14:48:56,049 - telegram.ext.updater - DEBUG - dispatcher - ended
2018-03-01 14:48:56,088 - telegram.ext.dispatcher - DEBUG - Waiting for async thread 1/4 to end
2018-03-01 14:48:56,088 - telegram.ext.dispatcher - DEBUG - Closing run_async thread 5a929e46-fac7-4a95-8da4-e4466149c7b6_0/4
2018-03-01 14:48:56,088 - telegram.ext.dispatcher - DEBUG - Closing run_async thread 5a929e46-fac7-4a95-8da4-e4466149c7b6_1/4
2018-03-01 14:48:56,089 - telegram.ext.dispatcher - DEBUG - Closing run_async thread 5a929e46-fac7-4a95-8da4-e4466149c7b6_2/4
2018-03-01 14:48:56,089 - telegram.e